https://www.kaggle.com/c/competitive-data-science-predict-future-sales/


In [28]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
data = pd.read_csv("./data/sales_train.csv")

In [4]:
data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [5]:
data = data.groupby(["date_block_num", "shop_id", "item_id"])["item_cnt_day"].sum().reset_index(name="item_cnt_month")

In [6]:
unique_shop_id = data["shop_id"].unique()
unique_item_id = data["item_id"].unique()
unique_date_block_num = data["date_block_num"].unique()

In [7]:
from itertools import product

In [8]:
date_block_shop_item_grid = pd.DataFrame([(shop_id, item_id, date_block) for shop_id, item_id, date_block in product(unique_shop_id, unique_item_id, unique_date_block_num)], columns=["shop_id", "item_id", "date_block_num"])

In [9]:
date_block_shop_item_grid = date_block_shop_item_grid.merge(data, on=["shop_id", "item_id", "date_block_num"], how="left")

In [10]:
date_block_shop_item_grid.fillna({"item_cnt_month": 0}, inplace=True)

In [11]:
date_block_shop_item_grid.head()

,shop_id,item_id,date_block_num,item_cnt_month
0,0,32,0,6.0
1,0,32,1,10.0
2,0,32,2,0.0
3,0,32,3,0.0
4,0,32,4,0.0


In [12]:
date_block_shop_item_grid.sort_values(by='date_block_num', inplace=True)

In [32]:
folds = [(list(range(25,31)),[31]), (list(range(26,32)),[32]), (list(range(27,33)),[33])]

In [33]:
from sklearn.metrics import mean_squared_error

In [34]:
test = pd.read_csv("./data/sales_test.csv")
test['date_block_num'] = 34
test.head()

,ID,shop_id,item_id,date_block_num
0,0,5,5037,34
1,1,5,5320,34
2,2,5,5233,34
3,3,5,5232,34
4,4,5,5268,34


## previuos period

In [35]:
metrics = []

In [36]:
for train_range, validation_range in  folds:
    train_data = date_block_shop_item_grid.loc[date_block_shop_item_grid['date_block_num'].isin(train_range)]
    valid_data =  date_block_shop_item_grid.loc[date_block_shop_item_grid['date_block_num'].isin(validation_range)]
    train_data['prev_value'] = train_data.groupby(['shop_id','item_id'])['item_cnt_month'].shift()
    train_data['prev_value'] = train_data['prev_value'].fillna(value=0)
    valid_data['prev_value'] = valid_data.groupby(['shop_id','item_id'])['item_cnt_month'].shift()
    valid_data['prev_value'] = valid_data['prev_value'].fillna(value=0)
    clipped_pred =  np.clip(valid_data['prev_value'], 0, 20)
    clipped_valid = np.clip(valid_data['item_cnt_month'], 0, 20)
    error = mean_squared_error(clipped_valid, clipped_pred)
    metrics.append(error)

In [37]:
metrics

[0.19874657984439248, 0.2342535271548891, 0.24714311918191406]

In [38]:
train_data['prev_value'] = train_data.groupby(['shop_id','item_id'])['item_cnt_month'].shift()

In [39]:
train_data[(train_data.shop_id == 2) & (train_data.item_id == 32)].sort_values(by=['shop_id', 'item_id', 'date_block_num']).head(300)

,shop_id,item_id,date_block_num,item_cnt_month,prev_value
1482903,2,32,27,0.0,NaN
1482904,2,32,28,0.0,0.0
1482905,2,32,29,0.0,0.0
1482906,2,32,30,0.0,0.0
1482907,2,32,31,1.0,0.0
1482908,2,32,32,0.0,1.0


In [40]:
prev_train = date_block_shop_item_grid[date_block_shop_item_grid.date_block_num == 33]

In [41]:
concated = pd.concat([prev_train, test])

In [42]:
concated['prediction'] = concated.groupby(['shop_id','item_id'])['item_cnt_month'].shift()

In [43]:
concated[(concated.shop_id == 2) & (concated.item_id == 32)].sort_values(by=['shop_id', 'item_id', 'date_block_num']).head(300)

,shop_id,item_id,date_block_num,item_cnt_month,ID,prediction
1482909,2,32,33,0.0,NaN,NaN
20995,2,32,34,NaN,20995.0,0.0


In [44]:
submission = concated[concated.date_block_num == 34][['ID', 'prediction']]
submission.rename(columns={"prediction": "item_cnt_month"}, inplace=True)
submission.reset_index(drop=True, inplace=True)
submission['item_cnt_month'] = submission['item_cnt_month'].fillna(value=0)
submission['item_cnt_month'] = np.clip(submission['item_cnt_month'], 0, 20)
submission

,ID,item_cnt_month
0,0.0,0.0
1,1.0,0.0
2,2.0,1.0
3,3.0,0.0
4,4.0,0.0
...,...,...
214195,214195.0,1.0
214196,214196.0,0.0
214197,214197.0,0.0
214198,214198.0,0.0


In [45]:
submission.astype({'ID': 'int32'}).to_csv('submission_sales.csv', index=False)

## prepare data for validation of mean and medain

In [46]:
def get_train_val_split(min_period_num = 0, max_periods_num = 33, number_of_train_periods=6, number_of_splits = None):
    n_split = 0
    folds = []
    rng = range(max_periods_num, min_period_num, -1)
    for i in rng:
        val_set = [i]
        train_start = i - number_of_train_periods
        trans_set = range(train_start, i) if train_start in rng else None
        if not trans_set:
            break
        folds.append((trans_set, val_set))
        n_split = n_split+1
        if n_split == number_of_splits:
            break
    folds.reverse()
    return folds

In [47]:
folds = get_train_val_split(number_of_splits=3, number_of_train_periods=6)
folds

[(range(25, 31), [31]), (range(26, 32), [32]), (range(27, 33), [33])]

## prepare data for prediction

In [50]:
train_folds_for_test_data = get_train_val_split(number_of_splits=1, number_of_train_periods=6, max_periods_num = 34)
train_data_range = train_folds_for_test_data[0][0]
validation_period = train_folds_for_test_data[0][1]
train_data = date_block_shop_item_grid.loc[date_block_shop_item_grid['date_block_num'].isin(train_data_range)]
train_data

,shop_id,item_id,date_block_num,item_cnt_month
37062510,17,21015,28,0.0
24445246,42,12124,28,0.0
24640032,43,9271,28,0.0
8827652,14,16655,28,0.0
39885598,40,4593,28,0.0
...,...,...,...,...
28810239,50,9420,33,0.0
28810273,50,10488,33,0.0
28810307,50,10719,33,0.0
28810069,50,2870,33,0.0


## mean n periods

In [165]:
metrics = []
for train_range, validation_range in  folds:
    train_data = date_block_shop_item_grid.loc[date_block_shop_item_grid['date_block_num'].isin(train_range)]
    valid_data =  date_block_shop_item_grid.loc[date_block_shop_item_grid['date_block_num'].isin(validation_range)]
    concated = pd.concat([train_data, valid_data])
    concated['prediction'] = concated.groupby(['shop_id','item_id'], sort=False)['item_cnt_month'].transform(lambda x: x.expanding().mean())
    valid_data_ext =  concated[concated.date_block_num == validation_range[0]]
    clipped_pred =  np.clip(valid_data_ext['prediction'], 0, 20)
    clipped_valid = np.clip(valid_data_ext['item_cnt_month'], 0, 20)
    error = mean_squared_error(clipped_valid, clipped_pred)
    metrics.append(error)
metrics

[0.11632737288064214, 0.14981119150095035, 0.16176887593667277]

In [51]:
concated = pd.concat([train_data, test])
concated

,shop_id,item_id,date_block_num,item_cnt_month,ID
37062510,17,21015,28,0.0,NaN
24445246,42,12124,28,0.0,NaN
24640032,43,9271,28,0.0,NaN
8827652,14,16655,28,0.0,NaN
39885598,40,4593,28,0.0,NaN
...,...,...,...,...,...
214195,45,18454,34,NaN,214195.0
214196,45,16188,34,NaN,214196.0
214197,45,15757,34,NaN,214197.0
214198,45,19648,34,NaN,214198.0


In [100]:
#concated['prediction'] = concated.groupby(['shop_id','item_id'])['item_cnt_month'].transform('mean')
concated['prediction'] = concated.groupby(['shop_id','item_id'], sort=False)['item_cnt_month'].transform(lambda x: x.expanding().mean())
concated

,shop_id,item_id,date_block_num,item_cnt_month,ID,prediction
27,0,32,27,0.0,NaN,0.000000
28,0,32,28,0.0,NaN,0.000000
29,0,32,29,0.0,NaN,0.000000
30,0,32,30,0.0,NaN,0.000000
31,0,32,31,0.0,NaN,0.000000
...,...,...,...,...,...,...
214195,45,18454,34,NaN,214195.0,0.142857
214196,45,16188,34,NaN,214196.0,0.000000
214197,45,15757,34,NaN,214197.0,0.000000
214198,45,19648,34,NaN,214198.0,0.000000


In [101]:
concated[(concated.shop_id == 2) & (concated.item_id == 32)].sort_values(by=['shop_id', 'item_id', 'date_block_num']).head(300)

,shop_id,item_id,date_block_num,item_cnt_month,ID,prediction
1482903,2,32,27,0.0,NaN,0.000000
1482904,2,32,28,0.0,NaN,0.000000
1482905,2,32,29,0.0,NaN,0.000000
1482906,2,32,30,0.0,NaN,0.000000
1482907,2,32,31,1.0,NaN,0.200000
1482908,2,32,32,0.0,NaN,0.166667
1482909,2,32,33,0.0,NaN,0.142857
20995,2,32,34,NaN,20995.0,0.142857


In [103]:
submission = concated[concated.date_block_num == validation_period][['ID', 'prediction']]
submission.rename(columns={"prediction": "item_cnt_month"}, inplace=True)
submission.reset_index(drop=True, inplace=True)
submission['item_cnt_month'] = submission['item_cnt_month'].fillna(value=0)
submission['item_cnt_month'] = np.clip(submission['item_cnt_month'], 0, 20)
submission

,ID,item_cnt_month
0,0.0,1.000000
1,1.0,0.000000
2,2.0,1.428571
3,3.0,0.142857
4,4.0,0.000000
...,...,...
214195,214195.0,0.142857
214196,214196.0,0.000000
214197,214197.0,0.000000
214198,214198.0,0.000000


In [104]:
submission.astype({'ID': 'int32'}).to_csv('submission_sales_mean.csv', index=False)

## medain n periods

In [166]:
metrics = []
for train_range, validation_range in  folds:
    train_data = date_block_shop_item_grid.loc[date_block_shop_item_grid['date_block_num'].isin(train_range)]
    valid_data =  date_block_shop_item_grid.loc[date_block_shop_item_grid['date_block_num'].isin(validation_range)]
    concated = pd.concat([train_data, valid_data])
    concated['prediction'] = concated.groupby(['shop_id','item_id'], sort=False)['item_cnt_month'].transform(lambda x: x.expanding().median())
    valid_data_ext =  concated[concated.date_block_num == validation_range[0]]
    clipped_pred =  np.clip(valid_data_ext['prediction'], 0, 20)
    clipped_valid = np.clip(valid_data_ext['item_cnt_month'], 0, 20)
    error = mean_squared_error(clipped_valid, clipped_pred)
    metrics.append(error)
metrics

[0.12045979119854482, 0.1733365433117806, 0.19436037358034883]

In [105]:
concated = pd.concat([train_data, test])
#concated['prediction'] = concated.groupby(['shop_id','item_id'])['item_cnt_month'].transform('median')
concated['prediction'] = concated.groupby(['shop_id','item_id'], sort=False)['item_cnt_month'].transform(lambda x: x.expanding().median())

In [106]:
concated[(concated.shop_id == 2) & (concated.item_id == 32)].sort_values(by=['shop_id', 'item_id', 'date_block_num']).head(300)

,shop_id,item_id,date_block_num,item_cnt_month,ID,prediction
1482902,2,32,26,1.0,NaN,1.0
1482903,2,32,27,0.0,NaN,0.5
1482904,2,32,28,0.0,NaN,0.0
1482905,2,32,29,0.0,NaN,0.0
1482906,2,32,30,0.0,NaN,0.0
1482907,2,32,31,1.0,NaN,0.0
1482908,2,32,32,0.0,NaN,0.0
1482909,2,32,33,0.0,NaN,0.0
20995,2,32,34,NaN,20995.0,0.0


In [107]:
concated[concated.prediction > 1]

,shop_id,item_id,date_block_num,item_cnt_month,ID,prediction
1487628,2,1905,26,2.0,NaN,2.0
1487629,2,1905,27,2.0,NaN,2.0
1487630,2,1905,28,1.0,NaN,2.0
1487631,2,1905,29,1.0,NaN,1.5
1488716,2,2308,26,2.0,NaN,2.0
...,...,...,...,...,...,...
209985,45,16227,34,NaN,209985.0,1.5
210001,45,1384,34,NaN,210001.0,1.5
210079,45,4894,34,NaN,210079.0,2.0
211167,45,1556,34,NaN,211167.0,1.5


In [108]:
submission = concated[concated.date_block_num == 34][['ID', 'prediction']]
submission.rename(columns={"prediction": "item_cnt_month"}, inplace=True)
submission.reset_index(drop=True, inplace=True)
submission['item_cnt_month'] = submission['item_cnt_month'].fillna(value=0)
submission['item_cnt_month'] = np.clip(submission['item_cnt_month'], 0, 20)
submission

,ID,item_cnt_month
0,0.0,1.0
1,1.0,0.0
2,2.0,1.0
3,3.0,0.0
4,4.0,0.0
...,...,...
214195,214195.0,0.0
214196,214196.0,0.0
214197,214197.0,0.0
214198,214198.0,0.0


In [109]:
submission.astype({'ID': 'int32'}).to_csv('submission_sales_median.csv', index=False)